In [1]:
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,"
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,"
    "org.postgresql:postgresql:42.5.0 pyspark-shell"
)

In [4]:
import pandas as pd

df = pd.read_csv("/home/lupusruber/music_analytics/dummy_dwh/SongDim.csv")

In [6]:
df.groupby("SongSK").size().reset_index(name="count")["count"].unique()

array([1])

In [3]:
from pyspark.sql.session import SparkSession

In [6]:
with SparkSession.builder.appName("KafkaStreaming").master(
    "local[*]"
).getOrCreate() as spark:
    DB_TABLE = "SongDim"
    DB_URL = f"jdbc:postgresql://localhost:5432/dwh"
    DB_PROPERTIES = {
        "user": "root",
        "password": "root",
        "driver": "org.postgresql.Driver",
    }

    df = spark.read.option("header", "true").csv(
        "/home/lupusruber/music_analytics/SongDimHeader.csv"
    )
    df.write.jdbc(
        url=DB_URL, table=DB_TABLE, mode="overwrite", properties=DB_PROPERTIES
    )
    spark.stop()